In [ ]:
pip install delta-spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-efbe2aa4-2636-433b-ac8b-f065b499dc31;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 938ms :: artifacts dl 21ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-run

### How to query a Delta Table from Presto/Trino?

#### From Semana 9 examples...
We have created a delta table `sales_per_country` that we would like to query through Presto/Trino.

`CREATE EXTERNAL TABLE gold_sales_demo.sales_per_country (
    year INT,
    country STRING,
    product_name STRING,
    product_color STRING,
    quantity INT, 
    total_price INT,
    unit_price_avg DOUBLE,
    unit_cost_avg DOUBLE,
    total_cost INT,
    revenue INT
)
USING DELTA
LOCATION 'hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/sales_per_country/'`

To be able to query it by other processing engines (that is, other than Apache Spark), we can generate a **manifest** file for the Delta table. Bellow we provide an example to generate a manifest file that can be used by Presto and Athena to read the Delta table.

The generate command generates manifest files at <path-to-delta-table>/_symlink_format_manifest/. In other words, the files in this directory will contain the names of the data files (that is, Parquet files) that should be read for reading a snapshot of the Delta table.

For more information please refer to: https://docs.databricks.com/delta/presto-integration.html

In [2]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/sales_per_country/`
""").show()

++
||
++
++



In [3]:
spark.sql("""
DROP TABLE IF EXISTS gold_sales_demo.sales_per_country_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS gold_sales_demo.sales_per_country_presto (
    year INT,
    country STRING,
    product_name STRING,
    product_color STRING,
    quantity INT, 
    total_price INT,
    unit_price_avg DOUBLE,
    unit_cost_avg DOUBLE,
    total_cost INT,
    revenue INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/sales_per_country/_symlink_format_manifest/'
""").show()

21/12/06 12:24:23 WARN DataSource: All paths were ignored:
  hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/sales_per_country/_symlink_format_manifest


++
||
++
++



21/12/06 12:24:29 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


++
||
++
++



In [19]:
spark.sql("DROP TABLE IF EXISTS gold_sales_demo.sales_per_country_conformmed_presto").show()

++
||
++
++

